In [ ]:
import numpy as np
w = np.array([
  [1., 2., 3.],
  [4., 5., 6.]
])
ans01 = w.copy() # numpy 텐서의 복사
print(ans01)
v = w[:, 1:]
print(v)

u = 2*v
print(u)

a = u[:, -1:] + 1
ans02 = a.copy()
print(ans02)

In [ ]:
# 여기에 템플릿을 복사하고 수정하여 코드를 완성합니다.
#

import numpy as np
w = np.array(
    [
        [1., 2.],
        [3., 4.]       
    ]
, dtype='float32')
x = np.array([5.,6.], dtype='float32')

n_iter = 1000
for i in range(n_iter):
    x = np.dot(w, x.T)
print(x)
print(x - x)    # inf를 연산하면 invalid값인 nan이 발생할 수 있습니다.

ans03 = 52    # n_iter의 최소치 기입 'float32'의 경우 
ans04 = 422   # n_iter의 최소치 기입 'float64'의 경우

In [ ]:
# 여기에 템플릿을 복사하고 수정하여 코드를 완성합니다.
#

from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model
import numpy as np

img_url = 'http://sipi.usc.edu/database/preview/misc/4.1.06.png'
img_url2 = 'http://sipi.usc.edu/database/preview/misc/4.1.07.png'
filename = 'tree.png'


#plt.subplot(121),plt.imshow(image),plt.title('original image')
#plt.subplot(122),plt.imshow(image_dst),plt.title('Mirror Affine')
#plt.show()

img_path = keras.utils.get_file(filename, img_url)
img_path2 = keras.utils.get_file(filename, img_url2)

src = cv2.imread(img_path)
src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)

src2 = cv2.imread(img_path2)
src2 = cv2.cvtColor(src2, cv2.COLOR_BGR2RGB)
# 영상을 80x80로 resize한다. (사이즈 정규화)
image = cv2.resize(src, dsize=(80, 80), interpolation=cv2.INTER_AREA)
image2 = cv2.resize(src2, dsize=(80, 80), interpolation=cv2.INTER_AREA)
# 입력 텐서의 정의: 값을 [0..1]사이로 정규화한다. image/255. (레벨 정규화)
image = image/255.
image2 = image2/255.

plt.figure(figsize=(10, 4))
plt.subplot(121),plt.imshow(image),plt.title('image 1')
plt.subplot(122),plt.imshow(image2),plt.title('image 2')
plt.show()

box_blur = np.array([[1, 1, 1],
                     [1, 1, 1],
                     [1, 1, 1] ])/9.

def my_init(shape, dtype=None):
    # 커널용 텐서의 정의: (3,3,1,1) = (3x3)커널, 1개의 입력 채널, 1개의 출력 채널
    kernel_data = np.reshape(box_blur, (3,3,1,1))
    # numpy용 텐서를 keras 텐서로 변환
    kernel = tf.constant(kernel_data, dtype='float32')
    return kernel

# 입력은 80x80 영상 컬러 이미지(3개의 r,g,b채널)
inputs = Input((80,80,3))
print('inputs.shape =', inputs.shape)

# 채널 분리
# inputs의 shape = (1, 80, 80, 3)
# inputs_r의 shape = (1, 80, 80, 1)
inputs_r = inputs[:,:,:,0:1] # data_number(=1), height(=80), width(=80), first_channel
inputs_g = inputs[:,:,:,1:2] # data_number(=1), height(=80), width(=80), second_channel
inputs_b = inputs[:,:,:,2:3] # data_number(=1), height(=80), width(=80), third_channel

# 컨볼루션 정의
# filters: 필터의 갯수 1개
# kernel_size: 3x3 필터를 사용한다.
# padding='same': 출력 텐서의 크기를 입력 텐서와 동일하게 유지한다. (가장자리를 0으로 채움)
# kernel_initializer: 커널의 초기값을 설정. 
# bias_initializer: bias는 0을 사용한다.
conv2d = Conv2D(filters=1, kernel_size=(3,3), padding='same', kernel_initializer=my_init, bias_initializer='zeros')

# 컨볼루션 처리
outputs_r = conv2d(inputs_r)
outputs_g = conv2d(inputs_g)
outputs_b = conv2d(inputs_b)

# 채널 병합
outputs = tf.concat([outputs_r, outputs_g, outputs_b], axis=3) # 채널 축으로 병합
print('outputs.shape =', outputs.shape)

# keras 모델 정의: 입력텐서와 출력텐서를 이용하여 정의한다.
model = Model(inputs, outputs)

# 데이터로 목록화 한다. 입력데이터가 1개이므로: [ 입력이미지1 ]
# 만일 입력데이터가 2개일 경우에는 : [ 입력이미지1, 입력이미지2 ]의 모양이 된다.
# 이 과정은 여러개의 이미지를 한번에 convolution 하게하기 위한 기본과정이다.
x = np.array([image, image2]) # make to data list

# 모델을 호출하여 컨볼루션 수행
out_images = model(x)

# 출력의 0번째에 첫번째 데이터가 저장되어 있다.
out_image = out_images[0].numpy()
out_image2 = out_images[1].numpy()

# 이미지 출력
plt.figure(figsize=(10, 4))
plt.subplot(121),plt.imshow(image),plt.title('output_image 1')
plt.subplot(122),plt.imshow(image2),plt.title('outpiut_image 2')
plt.show()


# 다음 코드는 첫번째 이미지 용입니다. 두번째 이미지로 변경하여야 합니다.
extract = x[1, 1:4, 1:4, 0]
print(extract)

avg = np.mean(extract)
print('9 pixles average =', avg)

# 다음 코드는 첫번째 이미지 용입니다. 두번째 이미지로 변경합니다.
conv2d_val = out_images[1, 2, 2, 0].numpy()
print('convolution result =', conv2d_val)

ans22 = conv2d_val

# 다음은 강의내용, 과제내용등에 대해 자유롭게 의견이나 느낀점을 적어주세요.
# 의견이 없으시면 공란으로 두셔도 무방합니다.

ans23 = """
딥러닝을 쉽게 알려주셔서 감사하게 수강하고 있습니다.
그럼에도 불구하고 내용 자체가 쉽지 않다보니 구글링을 통해 찾아가면서 학습을 하고 있는데요!
재미있어서 열심히 과제를 하게 되었지만 답안이나 해설이 따로 없어서 제가 맞게 했는지 잘 모르겠습니다.

과제 풀이 및 해석 or 실습에서 조금 더 자세하게 설명해주시면 감사할 것 같습니다.

감사합니다.
"""